In [1]:
!pip install langchain-groq
!pip install -U langchain-community
!pip install chromadb
!pip install pinecone

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
import pinecone
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain

In [4]:
loader = TextLoader("/content/menu_card.txt")
data = loader.load()
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

You have 1 document(s) in your data
There are 4236 characters in your sample document
Here is a sample: Restaurant Name: Gourmet Haven

Menu:

Item: Espresso Coffee
Ingredients: Finely ground coffee beans, water
Taste: Strong, bold, slightly bitter
Price: $2.50
Cooking Process: Hot water is forced throu


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [6]:
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 11 documents


In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-small-en",  # 1536 dim
    model_kwargs={"trust_remote_code": True}
)


# Assuming 'texts' is a list of Documents or strings
vectorstore2 = Chroma.from_documents(texts, embeddings)

<ipython-input-7-5d7244332445>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=userdata.get('GROQ_API_KEY')
)

In [9]:
chain = load_qa_chain(llm, chain_type="stuff")

<ipython-input-9-1b5424917bc0>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [10]:
query = "Do you have coffee?"
docs = vectorstore2.similarity_search(query)
chain.run(input_documents=docs, question=query)

<ipython-input-10-b4c6e408ee0f>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


'Yes, Gourmet Haven has a variety of coffee options available. They offer Cold Brew Coffee, Americano Coffee, Latte Coffee, Cappuccino Coffee, and Espresso Coffee.'

In [11]:
query = "What is the cheapest pizza?"
docs = vectorstore2.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The Margherita Pizza at Gourmet Haven is the cheapest pizza, priced at $8.99.'